In [91]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [92]:
df= pd.read_csv("data prep/datasets/2024_pregame.csv")

# X = df.drop("EI", axis=1)
# X=df.drop(["FGM_x","FGA_x","FG_PCT_x","FG3A_x","FG3_PCT_x","FTM_x","FTA_x","FT_PCT_x","OREB_x","DREB_x","FGM_y","FGA_y","FG_PCT_y","FG3A_y","FG3_PCT_y","FTM_y","FTA_y","FT_PCT_y","OREB_y","DREB_y","BLKA_x","PFD_x","BLKA_y","PFD_y","EI"], axis=1)

# X= df[["TIME","RIVALRY","NATIONAL_TV","DAY_Friday","DAY_Monday","DAY_Saturday","DAY_Sunday","DAY_Thursday","DAY_Tuesday","DAY_Wednesday","LEADERS_x","W_PCT_x","REB_x","AST_x","TOV_x","STL_x","BLK_x","PTS_x","PLUS_MINUS_x","LEADERS_y","W_PCT_y","REB_y","AST_y","TOV_y","STL_y","BLK_y","PTS_y","PLUS_MINUS_y"]]

# X= df[["TIME","RIVALRY","NATIONAL_TV","DAY_Friday","DAY_Monday","DAY_Saturday","DAY_Sunday","DAY_Thursday","DAY_Tuesday","DAY_Wednesday","LEADERS_x","W_PCT_x","REB_x","AST_x","TOV_x","STL_x","BLK_x","PTS_x","PLUS_MINUS_x","LEADERS_y","W_PCT_y","REB_y","AST_y","TOV_y","STL_y","BLK_y","PTS_y","PLUS_MINUS_y"]]

X= df[["TIME","RIVALRY","NATIONAL_TV","DAY_Friday","DAY_Monday","DAY_Saturday","DAY_Sunday","DAY_Thursday","DAY_Tuesday","DAY_Wednesday","LEADERS_x","W_PCT_x","PTS_x","PLUS_MINUS_x","LEADERS_y","W_PCT_y","PTS_y","PLUS_MINUS_y"]]

# X= X.iloc[:,6:]
y = df["EI"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [93]:
model = LinearRegression()
model.fit(X_train, y_train)

# model = RandomForestRegressor()
# model.fit(X_train, y_train)

# model = XGBRegressor()
# model.fit(X_train, y_train)

y_pred = model.predict(X_test)


print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))


MAE: 0.11586541098560049
RMSE: 0.13100124815650302
R²: 0.0018391546336248288


In [88]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Ridge & Lasso Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
ridge_preds = ridge.predict(X_test_scaled)

lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
lasso_preds = lasso.predict(X_test_scaled)

svr = SVR(kernel="rbf", C=1.0, epsilon=0.1)
svr.fit(X_train_scaled, y_train)
svr_preds = svr.predict(X_test_scaled)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))

NameError: name 'StandardScaler' is not defined